In [1]:
import sys, os

In [2]:
import sys, os
import numpy as np
import pandas as pd
import pickle

In [3]:
sys.path.append(os.path.abspath('../../'))
from utils import utils
from utils.utils import evaluate_experiment
sys.path.append(os.path.abspath('../'))
from timeseries_utils import *

In [4]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from scipy.fftpack import fft
from torch.autograd import Variable
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler

In [5]:
seed = 1
np.random.seed(seed)

## Printing configs

In [6]:
print('torch      :', torch.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
iscuda = torch.cuda.is_available()
if iscuda:
    print(f'device     : {torch.cuda.get_device_name(0)}')

torch      : 1.5.1
numpy      : 1.18.1
pandas     : 1.0.5
device     : GeForce GTX 1080


## Loading data

In [7]:
path_to_data='../../../'

In [8]:
data=np.load(os.path.abspath(path_to_data+'data-002.npy'),allow_pickle=True)
Y=np.load(path_to_data+'Y.npy',allow_pickle=True)
labels =pd.read_csv(path_to_data+'labels.csv')

In [9]:
train_fold=8
val_fold=9
test_fold=10

# 10th fold for testing (9th for now)
X_test = data[labels.strat_fold == test_fold]
y_test = Y[labels.strat_fold == test_fold]
# 9th fold for validation (8th for now)
X_val = data[labels.strat_fold == val_fold]
y_val = Y[labels.strat_fold == val_fold]
# rest for training
X_train = data[labels.strat_fold <= train_fold]
y_train = Y[labels.strat_fold <= train_fold]

In [10]:
X_train.shape ,X_val.shape

((17111, 1000, 12), (2156, 1000, 12))

## Preprocess signal data

In [11]:
X_train, X_val, X_test = utils.preprocess_signals(X_train, X_val, X_test,'/content/')
n_classes = y_train.shape[1]
X_train = np.reshape(X_train,[X_train.shape[0],X_train.shape[2],X_train.shape[1]])
X_val = np.reshape(X_val,[X_val.shape[0],X_val.shape[2],X_val.shape[1]])
X_test = np.reshape(X_test,[X_test.shape[0],X_test.shape[2],X_test.shape[1]])

In [12]:
X_train.shape, X_val.shape

((17111, 12, 1000), (2156, 12, 1000))

In [13]:
y_train.shape, y_val.shape

((17111, 5), (2156, 5))

In [14]:
y_val_eval = y_val

## Get fourier transform

In [15]:
display(y_val.shape)
y_val_list = np.asarray([np.where(r==1)[0][0] for r in y_val])
y_train_list = np.asarray([np.where(r==1)[0][0] for r in y_train])
y_val_list.shape

(2156, 5)

(2156,)

In [16]:
N=1000
X_train_fft = fft(X_train)
X_train_fft = (2/N) * np.abs(X_train_fft)

X_val_fft = fft(X_val)
X_val_fft = (2/N) * np.abs(X_val_fft)
X_train, X_val = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train, X_val)]
X_train_fft, X_val_fft = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train_fft, X_val_fft)]
y_train_tensor, y_val_tensor = [torch.tensor(arr, dtype=torch.long) for arr in (y_train_list, y_val_list)]
train_ds = TensorDataset(X_train, X_train_fft, y_train_tensor)
valid_ds = TensorDataset(X_val, X_val_fft, y_val_tensor)

X_train.shape, X_train_fft.shape, y_train.shape

(torch.Size([17111, 12, 1000]), torch.Size([17111, 12, 1000]), (17111, 5))

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#batch size
bs=256
jobs=1
train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=jobs)
valid_dl = DataLoader(valid_ds, bs, shuffle=False, num_workers=jobs)

## Defining blocks

In [18]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.
    
    The separable convlution is a method to reduce number of the parameters 
    in the deep learning network for slight decrease in predictions quality.
    """
    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.
    
    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """
    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):
    
        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x): 
        return self.layers(x)


class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)
    
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

## Creating the Branched Model

In [19]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
class Classifier(nn.Module):
    #https://www.kaggle.com/purplejester/pytorch-deep-time-series-classification
    def __init__(self, raw_ni, fft_ni, no, drop=.5):
        super().__init__()
        
        self.raw = nn.Sequential(
            SepConv1d(raw_ni,  32, 5, 2, 3, drop=drop),
            SepConv1d(    32,  64, 5, 4, 2, drop=drop),
            SepConv1d(    64, 128, 5, 4, 2, drop=drop),
            SepConv1d(   128, 256, 5, 4, 2),
            Flatten(),
            nn.Dropout(drop), nn.Linear(2048, 64), nn.ReLU(inplace=True), # here was 256,64
            nn.Dropout(drop), nn.Linear(64, 64), nn.ReLU(inplace=True))
        
        self.fft = nn.Sequential(
            SepConv1d(fft_ni,  32, 5, 2, 3, drop=drop),
            SepConv1d(    32,  64, 5, 4, 2, drop=drop),
            SepConv1d(    64, 128, 5, 4, 2, drop=drop),
            SepConv1d(   128, 128, 5, 4, 2, drop=drop),
            SepConv1d(   128, 256, 5, 4, 2),
            Flatten(),
            nn.Dropout(drop), nn.Linear(512, 64), nn.ReLU(inplace=True), # here was 256,64
            nn.Dropout(drop), nn.Linear(64, 64), nn.ReLU(inplace=True))
        
        self.out = nn.Sequential(
            nn.Linear(128, 64), nn.ReLU(inplace=True), nn.Linear(64, no))
        
    def forward(self, t_raw, t_fft):
        raw_out = self.raw(t_raw)
        fft_out = self.fft(t_fft)
        t_in = torch.cat([raw_out, fft_out], dim=1)
        out = self.out(t_in)
        return out

## Training

In [20]:
lr = 0.01
#lr = 0.0005
n_epochs = 3000
iterations_per_epoch = len(train_ds)
num_classes = 5
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

display((X_train.shape, X_train_fft.shape))
model = Classifier(X_train.shape[1], X_train_fft.shape[1], num_classes).to(device)
#model = fcn_wang(num_classes=5,input_channels=X_train.shape[1]).to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

#criterion = nn.CrossEntropyLoss()
#opt = torch.optim.RMSprop(model.parameters(), lr=lr)
#sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

(torch.Size([17111, 12, 1000]), torch.Size([17111, 12, 1000]))

In [21]:
print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(train_dl):
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        
        opt.zero_grad()

        out = model(x_raw, x_fft)
        
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        #sched.step()
        
    epoch_loss /= len(train_ds)
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    for batch in valid_dl:
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        out = model(x_raw, x_fft)
        
        preds = F.softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    #if epoch % base == 0:
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
    #base *= step

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 1.4611. Acc.: 42.67%
Epoch 1 best model saved with accuracy: 42.67%
Epoch:   2. Loss: 1.4449. Acc.: 42.67%
Epoch:   3. Loss: 1.4255. Acc.: 46.20%
Epoch 3 best model saved with accuracy: 46.20%
Epoch:   4. Loss: 1.3645. Acc.: 48.52%
Epoch 4 best model saved with accuracy: 48.52%
Epoch:   5. Loss: 1.3423. Acc.: 49.30%
Epoch 5 best model saved with accuracy: 49.30%
Epoch:   6. Loss: 1.3296. Acc.: 49.07%
Epoch:   7. Loss: 1.3186. Acc.: 49.77%
Epoch 7 best model saved with accuracy: 49.77%
Epoch:   8. Loss: 1.3143. Acc.: 49.63%
Epoch:   9. Loss: 1.3130. Acc.: 49.68%
Epoch:  10. Loss: 1.3089. Acc.: 49.26%
Epoch:  11. Loss: 1.3051. Acc.: 49.21%
Epoch:  12. Loss: 1.3032. Acc.: 50.32%
Epoch 12 best model saved with accuracy: 50.32%
Epoch:  13. Loss: 1.2969. Acc.: 50.42%
Epoch 13 best model saved with accuracy: 50.42%
Epoch:  14. Loss: 1.2958. Acc.: 50.56%
Epoch 14 best model saved with accuracy: 50.56%
Epoch:  15. Loss: 1.2902. Acc.: 50.32%
Epoch:  16. Los

Epoch: 181. Loss: 0.9893. Acc.: 56.26%
Epoch: 182. Loss: 0.9938. Acc.: 56.49%
Epoch: 183. Loss: 0.9926. Acc.: 57.19%
Epoch: 184. Loss: 0.9909. Acc.: 55.98%
Epoch: 185. Loss: 0.9843. Acc.: 57.28%
Epoch: 186. Loss: 0.9841. Acc.: 56.31%
Epoch: 187. Loss: 0.9878. Acc.: 56.22%
Epoch: 188. Loss: 0.9843. Acc.: 55.89%
Epoch: 189. Loss: 0.9844. Acc.: 55.38%
Epoch: 190. Loss: 0.9828. Acc.: 55.75%
Epoch: 191. Loss: 0.9803. Acc.: 56.54%
Epoch: 192. Loss: 0.9788. Acc.: 55.66%
Epoch: 193. Loss: 0.9803. Acc.: 56.22%
Epoch: 194. Loss: 0.9897. Acc.: 56.59%
Epoch: 195. Loss: 0.9856. Acc.: 56.54%
Epoch: 196. Loss: 0.9774. Acc.: 56.35%
Epoch: 197. Loss: 0.9836. Acc.: 55.15%
Epoch: 198. Loss: 0.9808. Acc.: 55.66%
Epoch: 199. Loss: 0.9828. Acc.: 55.24%
Epoch: 200. Loss: 0.9843. Acc.: 55.80%
Epoch: 201. Loss: 0.9759. Acc.: 56.96%
Epoch: 202. Loss: 0.9854. Acc.: 56.17%
Epoch: 203. Loss: 0.9815. Acc.: 55.29%
Epoch: 204. Loss: 0.9776. Acc.: 56.86%
Epoch: 205. Loss: 0.9849. Acc.: 55.57%
Epoch: 206. Loss: 0.9800.

Epoch: 390. Loss: 0.9340. Acc.: 55.84%
Epoch: 391. Loss: 0.9299. Acc.: 55.61%
Epoch: 392. Loss: 0.9342. Acc.: 56.17%
Epoch: 393. Loss: 0.9323. Acc.: 56.08%
Epoch: 394. Loss: 0.9330. Acc.: 56.08%
Epoch: 395. Loss: 0.9300. Acc.: 56.22%
Epoch: 396. Loss: 0.9287. Acc.: 55.80%
Epoch: 397. Loss: 0.9350. Acc.: 55.19%
Epoch: 398. Loss: 0.9324. Acc.: 56.03%
Epoch: 399. Loss: 0.9321. Acc.: 55.66%
Epoch: 400. Loss: 0.9277. Acc.: 55.75%
Epoch: 401. Loss: 0.9342. Acc.: 55.75%
Epoch: 402. Loss: 0.9309. Acc.: 55.38%
Epoch: 403. Loss: 0.9288. Acc.: 55.98%
Epoch: 404. Loss: 0.9259. Acc.: 55.38%
Epoch: 405. Loss: 0.9305. Acc.: 55.52%
Epoch: 406. Loss: 0.9347. Acc.: 55.61%
Epoch: 407. Loss: 0.9303. Acc.: 55.98%
Epoch: 408. Loss: 0.9253. Acc.: 55.61%
Epoch: 409. Loss: 0.9246. Acc.: 55.89%
Epoch: 410. Loss: 0.9334. Acc.: 55.52%
Epoch: 411. Loss: 0.9290. Acc.: 55.33%
Epoch: 412. Loss: 0.9289. Acc.: 55.24%
Epoch: 413. Loss: 0.9333. Acc.: 56.08%
Epoch: 414. Loss: 0.9285. Acc.: 56.03%
Epoch: 415. Loss: 0.9325.

Epoch: 599. Loss: 0.9399. Acc.: 55.75%
Epoch: 600. Loss: 0.9338. Acc.: 55.84%
Epoch: 601. Loss: 0.9309. Acc.: 57.28%
Epoch: 602. Loss: 0.9385. Acc.: 56.45%
Epoch: 603. Loss: 0.9363. Acc.: 57.37%
Epoch: 604. Loss: 0.9405. Acc.: 56.82%
Epoch: 605. Loss: 0.9356. Acc.: 56.17%
Epoch: 606. Loss: 0.9297. Acc.: 57.33%
Epoch: 607. Loss: 0.9356. Acc.: 55.98%
Epoch: 608. Loss: 0.9283. Acc.: 55.84%
Epoch: 609. Loss: 0.9319. Acc.: 56.08%
Epoch: 610. Loss: 0.9333. Acc.: 55.15%
Epoch: 611. Loss: 0.9377. Acc.: 54.92%
Epoch: 612. Loss: 0.9385. Acc.: 56.68%
Epoch: 613. Loss: 0.9286. Acc.: 56.54%
Epoch: 614. Loss: 0.9317. Acc.: 56.91%
Epoch: 615. Loss: 0.9331. Acc.: 54.92%
Epoch: 616. Loss: 0.9410. Acc.: 55.84%
Epoch: 617. Loss: 0.9380. Acc.: 56.86%
Epoch: 618. Loss: 0.9343. Acc.: 56.35%
Epoch: 619. Loss: 0.9296. Acc.: 56.96%
Epoch: 620. Loss: 0.9399. Acc.: 55.61%
Epoch: 621. Loss: 0.9324. Acc.: 56.22%
Epoch: 622. Loss: 0.9282. Acc.: 57.61%
Epoch: 623. Loss: 0.9293. Acc.: 56.35%
Epoch: 624. Loss: 0.9313.

Epoch: 807. Loss: 0.8861. Acc.: 57.93%
Epoch: 808. Loss: 0.8937. Acc.: 57.56%
Epoch: 809. Loss: 0.8844. Acc.: 57.14%
Epoch: 810. Loss: 0.8910. Acc.: 56.63%
Epoch: 811. Loss: 0.8907. Acc.: 57.28%
Epoch: 812. Loss: 0.8938. Acc.: 57.42%
Epoch: 813. Loss: 0.8881. Acc.: 57.14%
Epoch: 814. Loss: 0.8960. Acc.: 56.59%
Epoch: 815. Loss: 0.8881. Acc.: 57.33%
Epoch: 816. Loss: 0.8874. Acc.: 56.91%
Epoch: 817. Loss: 0.8858. Acc.: 56.77%
Epoch: 818. Loss: 0.8870. Acc.: 57.37%
Epoch: 819. Loss: 0.8870. Acc.: 56.96%
Epoch: 820. Loss: 0.8761. Acc.: 56.35%
Epoch: 821. Loss: 0.8871. Acc.: 56.86%
Epoch: 822. Loss: 0.8931. Acc.: 56.73%
Epoch: 823. Loss: 0.8887. Acc.: 57.14%
Epoch: 824. Loss: 0.8815. Acc.: 57.19%
Epoch: 825. Loss: 0.8799. Acc.: 56.82%
Epoch: 826. Loss: 0.8824. Acc.: 56.68%
Epoch: 827. Loss: 0.8837. Acc.: 56.82%
Epoch: 828. Loss: 0.8884. Acc.: 56.96%
Epoch: 829. Loss: 0.8829. Acc.: 56.86%
Epoch: 830. Loss: 0.8813. Acc.: 57.00%
Epoch: 831. Loss: 0.8822. Acc.: 57.33%
Epoch: 832. Loss: 0.8867.

...

Epoch: 1625. Loss: 0.7971. Acc.: 60.53%
Epoch: 1626. Loss: 0.7944. Acc.: 59.51%
Epoch: 1627. Loss: 0.7870. Acc.: 59.65%
Epoch: 1628. Loss: 0.8036. Acc.: 59.46%
Epoch: 1629. Loss: 0.8001. Acc.: 61.09%
Epoch: 1630. Loss: 0.8068. Acc.: 59.65%
Epoch: 1631. Loss: 0.7917. Acc.: 60.58%
Epoch: 1632. Loss: 0.7960. Acc.: 61.27%
Epoch: 1633. Loss: 0.8031. Acc.: 60.62%
Epoch: 1634. Loss: 0.7965. Acc.: 59.65%
Epoch: 1635. Loss: 0.8008. Acc.: 60.16%
Epoch: 1636. Loss: 0.7959. Acc.: 62.06%
Epoch 1636 best model saved with accuracy: 62.06%
Epoch: 1637. Loss: 0.8002. Acc.: 60.67%
Epoch: 1638. Loss: 0.7966. Acc.: 59.83%
Epoch: 1639. Loss: 0.8040. Acc.: 58.81%
Epoch: 1640. Loss: 0.7960. Acc.: 60.58%
Epoch: 1641. Loss: 0.7960. Acc.: 60.48%
Epoch: 1642. Loss: 0.7974. Acc.: 60.76%
Epoch: 1643. Loss: 0.7948. Acc.: 61.04%
Epoch: 1644. Loss: 0.7958. Acc.: 59.65%
Epoch: 1645. Loss: 0.7949. Acc.: 59.69%
Epoch: 1646. Loss: 0.7833. Acc.: 60.67%
Epoch: 1647. Loss: 0.7974. Acc.: 59.46%
Epoch: 1648. Loss: 0.7909. Acc

Epoch: 1830. Loss: 0.7526. Acc.: 61.04%
Epoch: 1831. Loss: 0.7561. Acc.: 60.76%
Epoch: 1832. Loss: 0.7497. Acc.: 61.32%
Epoch: 1833. Loss: 0.7548. Acc.: 60.67%
Epoch: 1834. Loss: 0.7568. Acc.: 61.41%
Epoch: 1835. Loss: 0.7550. Acc.: 61.50%
Epoch: 1836. Loss: 0.7536. Acc.: 61.09%
Epoch: 1837. Loss: 0.7524. Acc.: 60.71%
Epoch: 1838. Loss: 0.7561. Acc.: 61.50%
Epoch: 1839. Loss: 0.7578. Acc.: 60.58%
Epoch: 1840. Loss: 0.7533. Acc.: 61.36%
Epoch: 1841. Loss: 0.7487. Acc.: 61.18%
Epoch: 1842. Loss: 0.7516. Acc.: 61.41%
Epoch: 1843. Loss: 0.7513. Acc.: 61.36%
Epoch: 1844. Loss: 0.7518. Acc.: 60.67%
Epoch: 1845. Loss: 0.7430. Acc.: 60.85%
Epoch: 1847. Loss: 0.7504. Acc.: 61.09%
Epoch: 1848. Loss: 0.7538. Acc.: 60.81%
Epoch: 1849. Loss: 0.7440. Acc.: 61.32%
Epoch: 1850. Loss: 0.7477. Acc.: 60.58%
Epoch: 1851. Loss: 0.7447. Acc.: 60.90%
Epoch: 1852. Loss: 0.7585. Acc.: 61.18%
Epoch: 1853. Loss: 0.7451. Acc.: 61.13%
Epoch: 1854. Loss: 0.7428. Acc.: 60.99%
Epoch: 1855. Loss: 0.7565. Acc.: 60.85%


Epoch: 2036. Loss: 0.7332. Acc.: 60.90%
Epoch: 2037. Loss: 0.7395. Acc.: 60.85%
Epoch: 2038. Loss: 0.7344. Acc.: 60.85%
Epoch: 2039. Loss: 0.7328. Acc.: 60.81%
Epoch: 2040. Loss: 0.7345. Acc.: 60.90%
Epoch: 2041. Loss: 0.7376. Acc.: 60.90%
Epoch: 2042. Loss: 0.7301. Acc.: 60.76%
Epoch: 2043. Loss: 0.7295. Acc.: 60.85%
Epoch: 2044. Loss: 0.7515. Acc.: 60.44%
Epoch: 2045. Loss: 0.7590. Acc.: 60.99%
Epoch: 2046. Loss: 0.7536. Acc.: 60.53%
Epoch: 2047. Loss: 0.7647. Acc.: 60.71%
Epoch: 2048. Loss: 0.7564. Acc.: 60.90%
Epoch: 2049. Loss: 0.7636. Acc.: 60.16%
Epoch: 2050. Loss: 0.7759. Acc.: 60.85%
Epoch: 2051. Loss: 0.7835. Acc.: 59.65%
Epoch: 2052. Loss: 0.7718. Acc.: 59.79%
Epoch: 2053. Loss: 0.7711. Acc.: 60.67%
Epoch: 2054. Loss: 0.7786. Acc.: 60.44%
Epoch: 2055. Loss: 0.7753. Acc.: 60.34%
Epoch: 2056. Loss: 0.7656. Acc.: 60.53%
Epoch: 2057. Loss: 0.7734. Acc.: 60.62%
Epoch: 2058. Loss: 0.7749. Acc.: 61.36%
Epoch: 2059. Loss: 0.7724. Acc.: 61.69%
Epoch: 2060. Loss: 0.7568. Acc.: 61.13%


Epoch: 2241. Loss: 0.7481. Acc.: 59.65%
Epoch: 2242. Loss: 0.7437. Acc.: 61.55%
Epoch: 2243. Loss: 0.7495. Acc.: 60.34%
Epoch: 2244. Loss: 0.7345. Acc.: 60.44%
Epoch: 2245. Loss: 0.7453. Acc.: 60.39%
Epoch: 2246. Loss: 0.7395. Acc.: 60.85%
Epoch: 2247. Loss: 0.7521. Acc.: 62.01%
Epoch: 2248. Loss: 0.7468. Acc.: 60.81%
Epoch: 2249. Loss: 0.7466. Acc.: 60.99%
Epoch: 2250. Loss: 0.7467. Acc.: 61.09%
Epoch: 2251. Loss: 0.7464. Acc.: 60.34%
Epoch: 2252. Loss: 0.7479. Acc.: 61.22%
Epoch: 2253. Loss: 0.7456. Acc.: 62.06%
Epoch: 2254. Loss: 0.7419. Acc.: 60.76%
Epoch: 2255. Loss: 0.7421. Acc.: 60.76%
Epoch: 2256. Loss: 0.7453. Acc.: 60.81%
Epoch: 2257. Loss: 0.7316. Acc.: 60.99%
Epoch: 2258. Loss: 0.7411. Acc.: 60.48%
Epoch: 2259. Loss: 0.7475. Acc.: 59.97%
Epoch: 2260. Loss: 0.7387. Acc.: 61.50%
Epoch: 2261. Loss: 0.7415. Acc.: 60.06%
Epoch: 2262. Loss: 0.7418. Acc.: 60.44%
Epoch: 2263. Loss: 0.7439. Acc.: 61.50%
Epoch: 2264. Loss: 0.7368. Acc.: 61.09%
Epoch: 2265. Loss: 0.7301. Acc.: 61.04%


Epoch: 2446. Loss: 0.7129. Acc.: 60.44%
Epoch: 2447. Loss: 0.7240. Acc.: 60.76%
Epoch: 2448. Loss: 0.7142. Acc.: 60.67%
Epoch: 2449. Loss: 0.7030. Acc.: 60.25%
Epoch: 2450. Loss: 0.7131. Acc.: 60.48%
Epoch: 2451. Loss: 0.7169. Acc.: 61.09%
Epoch: 2452. Loss: 0.7190. Acc.: 60.62%
Epoch: 2453. Loss: 0.7114. Acc.: 60.53%
Epoch: 2454. Loss: 0.7177. Acc.: 60.67%
Epoch: 2455. Loss: 0.7146. Acc.: 60.99%
Epoch: 2456. Loss: 0.7035. Acc.: 60.48%
Epoch: 2457. Loss: 0.7179. Acc.: 60.71%
Epoch: 2458. Loss: 0.7125. Acc.: 60.85%
Epoch: 2459. Loss: 0.7155. Acc.: 61.13%
Epoch: 2460. Loss: 0.7130. Acc.: 60.99%
Epoch: 2461. Loss: 0.7150. Acc.: 60.90%
Epoch: 2462. Loss: 0.7069. Acc.: 60.67%
Epoch: 2463. Loss: 0.7130. Acc.: 60.62%
Epoch: 2464. Loss: 0.7073. Acc.: 60.48%
Epoch: 2465. Loss: 0.7021. Acc.: 60.76%
Epoch: 2466. Loss: 0.7121. Acc.: 60.53%
Epoch: 2467. Loss: 0.7076. Acc.: 60.62%
Epoch: 2468. Loss: 0.7211. Acc.: 60.76%
Epoch: 2469. Loss: 0.7138. Acc.: 60.95%
Epoch: 2470. Loss: 0.7116. Acc.: 60.81%


## Evaluating

In [22]:
preds = []
y_eval = []
print('Predicting on validation dataset')
for batch in valid_dl:
    x_raw, x_fft, y_batch = [t.to(device) for t in batch]
    out = model(x_raw, x_fft)
    #print(out.view())
    preds += F.softmax(out, dim=1).tolist()
preds = np.array(preds,dtype=np.float32)

Predicting on validation dataset


In [23]:
from utils.utils import evaluate_experiment

In [24]:
tr_df_point = evaluate_experiment(np.array(y_val_eval,dtype=np.float32), np.array(preds,dtype=np.float32))
print(tr_df_point)

   macro_auc      Fmax
0   0.834179  0.697502


# To test: add a branch with wavelet

# To test: Search for something like mel

# To test: add a branch with images